In [2]:
#Load the needed Packages
import xarray as xr
import numpy as np
import datetime
import utm
import matplotlib.pyplot as plt
import rasterio.mask
import geopandas as gpd
import pandas as pd
import glob
import os.path
from shapely.geometry import mapping
import time

In [3]:
shapefile = '/Users/shahabshojaee/Documents/All_stations_with_NHD/HUC_splitted/*.shp'
path = '/Users/shahabshojaee/Documents/All_stations_with_NHD/daymet_data/*_na.nc4'
root = '/Users/shahabshojaee/Documents/All_stations_with_NHD/daymet_HUC/'
datasets_nc = [f for f in glob.glob(path)] 
datasets_shp = [f for f in glob.glob(shapefile)] 

In [4]:
src0 = []
for j in range(len(datasets_nc)):
    src0.append(rasterio.open(datasets_nc[j]))
        
        

In [ ]:
start_date = datetime.datetime(1980, 1, 1)

start = time.time()
for k in range(len(datasets_shp)):
    
    if os.path.exists(datasets_shp[k][:-4] + '.nc4') != True:
        values = []
        for j in range(len(src0)):

            #src0 = rasterio.open(datasets_nc[j]) 
            shapefile = gpd.read_file(datasets_shp[k])
            shapefile = shapefile.to_crs(crs=src0[j].crs.data)
            geoms = shapefile.geometry.values # list of shapely geometries
            if len(geoms) != 0:
                geometry = geoms[0] # shapely geometry
                # transform to GeJSON format

                geoms = [mapping(geoms[0])]

                
                out_image, out_transform = rasterio.mask.mask(src0[j], geoms, crop=True)
                np.place(out_image,out_image == -9999, np.nan)
                dss = xr.DataArray(out_image)





                for i in range(len(dss)):
                    dp = dss[i]
                    a = {}
                    a = {'Min': dp.min(skipna = True).values,
                        'P1': dp.quantile(.01).values,
                        'P5': dp.quantile(.05).values,
                        'P10': dp.quantile(.1).values,
                        'P20': dp.quantile(.2).values,
                        'P25': dp.quantile(.25).values,
                        'P30': dp.quantile(.3).values,
                        'P40': dp.quantile(.4).values,
                        'P50': dp.quantile(.5).values,
                        'P60': dp.quantile(.6).values,
                        'P70': dp.quantile(.7).values,
                        'P75': dp.quantile(.75).values,
                        'P80': dp.quantile(.8).values,
                        'P90': dp.quantile(.9).values,
                        'P95': dp.quantile(.95).values,
                        'P99': dp.quantile(.99).values,
                        'Mean' : dp.mean(skipna = True).values,
                        'Max': dp.max(skipna = True).values,
                        'Var': dp.var(skipna = True).values,
                        'Std': dp.std(skipna = True).values}

                    values.append(a)



        df = pd.DataFrame(values)
        dfp = df.to_xarray()
        dt_array = np.array([start_date + datetime.timedelta(days=i) for i in range(len(dfp.index))])
        dfp['index'] = dt_array
        dfp = dfp.rename({'index':'time'})
        dfp.to_netcdf(datasets_shp[k][:-4] + '.nc4')
        end = time.time()
        print(end - start)


503.9113049507141
1013.0460338592529
1891.358943939209
2301.8033289909363
3115.683163881302
3476.9680919647217
3885.9265229701996
4365.596359968185
4737.577724933624
5105.358449935913
5588.036670923233
5940.451148748398
6435.617511987686
6892.971899986267
7366.012043952942
7840.083639860153
8208.921734809875
9138.155761957169
10142.03779888153
11105.579717874527
11595.708387851715
12068.807647943497
12494.129477977753
13346.684068918228
13831.975723981857
14307.31377696991
14781.212359905243
15259.872298002243
15729.572530984879
16097.670021772385
16976.96207690239
17736.687587738037
18214.399339914322
18680.17230296135
19165.388044834137
19660.71759700775
20160.07322382927
20644.55692076683
21122.11715888977
21589.322021961212
22485.977432727814
22955.000403881073
23937.88772392273
24355.359446763992
24863.426035881042
25338.255934000015
25844.57158780098
26315.139820814133
26733.21337389946
27283.70045900345
27811.946238040924
28299.89915394783
29278.658278942108
29788.52534389496
30

In [5]:
src0

[<open DatasetReader name='/Users/shahabshojaee/Documents/All_stations_with_NHD/daymet_data/daymet_prcp1983_na.nc4' mode='r'>,
 <open DatasetReader name='/Users/shahabshojaee/Documents/All_stations_with_NHD/daymet_data/daymet_prcp1991_na.nc4' mode='r'>,
 <open DatasetReader name='/Users/shahabshojaee/Documents/All_stations_with_NHD/daymet_data/daymet_prcp2017_na.nc4' mode='r'>,
 <open DatasetReader name='/Users/shahabshojaee/Documents/All_stations_with_NHD/daymet_data/daymet_prcp2009_na.nc4' mode='r'>,
 <open DatasetReader name='/Users/shahabshojaee/Documents/All_stations_with_NHD/daymet_data/daymet_prcp2005_na.nc4' mode='r'>,
 <open DatasetReader name='/Users/shahabshojaee/Documents/All_stations_with_NHD/daymet_data/daymet_prcp2007_na.nc4' mode='r'>,
 <open DatasetReader name='/Users/shahabshojaee/Documents/All_stations_with_NHD/daymet_data/daymet_prcp2015_na.nc4' mode='r'>,
 <open DatasetReader name='/Users/shahabshojaee/Documents/All_stations_with_NHD/daymet_data/daymet_prcp1993_na.